Installation de Keras

In [1]:
!pip install -U keras

In [2]:
# utilisons pytorch plutôt que tensorflow
import os
os.environ["KERAS_BACKEND"] = "torch"

Exploration du jeu de donnée

In [3]:
import keras
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
print("Dimension de x_train:", x_train.shape)
print(x_train.shape[0], "images d'entrainement")
print(x_test.shape[0], "images de test")

Dimension de x_train: (60000, 28, 28)
60000 images d'entrainement
10000 images de test


In [5]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [6]:
from keras.utils import to_categorical

# nombre de classe de notre modèle
num_classes = 10

# on mets les données en forme avec l'encodage "one-hot"
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

Création du modèle

In [7]:
from keras import layers

In [39]:
# Réseau de neurones entièrement connecté
'''
model = keras.Sequential(
    [
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(400, activation="relu"),
        layers.Dense(200, activation="relu"),
        layers.Dense(100, activation="relu"),
        layers.Dense(75, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
'''

'\nmodel = keras.Sequential(\n    [\n        layers.Flatten(input_shape=(28, 28)),\n        layers.Dense(400, activation="relu"),\n        layers.Dense(200, activation="relu"),\n        layers.Dense(100, activation="relu"),\n        layers.Dense(75, activation="relu"),\n        layers.Dense(num_classes, activation="softmax"),\n    ]\n)\n'

In [8]:
# Alternative: réseau de neurones à convolution
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [9]:
# "résumé" de notre modèle
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │          16,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(
    loss="categorical_crossentropy", # on veut optimiser la capacité de catégorisation du modèle
    optimizer="adam", # le "moteur" utilisé pour ajuster les paramètres
    metrics=["accuracy"] # ce qu'on veut monitorer: la "précision" du modèle
)

In [11]:
BATCH_SIZE = 256

In [12]:
hist = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=5,
    validation_split=0.1
)

Epoch 1/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6932 - loss: 1.0174 - val_accuracy: 0.9725 - val_loss: 0.1085
Epoch 2/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9528 - loss: 0.1581 - val_accuracy: 0.9807 - val_loss: 0.0745
Epoch 3/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9682 - loss: 0.1070 - val_accuracy: 0.9853 - val_loss: 0.0598
Epoch 4/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9745 - loss: 0.0857 - val_accuracy: 0.9852 - val_loss: 0.0540
Epoch 5/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9771 - loss: 0.0753 - val_accuracy: 0.9875 - val_loss: 0.0502


In [13]:
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9813 - loss: 0.0506


[0.04210102930665016, 0.9855999946594238]

In [14]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.04210102930665016
Test accuracy: 0.9855999946594238


In [47]:
model.save("model_5couches.keras")